In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 4)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model_ft.load_state_dict(torch.load('/content/drive/MyDrive/Plant_Pathology/resnet18_f.pth'))

<All keys matched successfully>

In [ ]:
model_ft.eval()

In [ ]:
class ppDataset(torch.utils.data.Dataset):
    def __init__(self, df, image_dir, return_labels=False, transforms=None):
        self.df = df
        self.image_dir = image_dir
        self.transforms = transforms
        self.return_labels = return_labels
        self.label_map = {'healthy':0, 'multiple_diseases':1, 'rust':2, 'scab':3}
        self.label_map_reverse = {v:k for k,v in self.label_map.items()}
        
    def __len__(self):
        return self.df.__len__()
    
    def __getitem__(self, idx):
        image_path = self.image_dir + self.df.loc[idx, 'image_id'] + '.jpg'
        image = Image.open(image_path).convert('RGB')
        
        if self.transforms:
            image = self.transforms(image)

        if self.return_labels:
            # label = torch.tensor(self.df.loc[idx, ['healthy', 'multiple_diseases', 'rust', 'scab']]).unsqueeze(1)
            label = torch.tensor(self.df.loc[idx, ['healthy', 'multiple_diseases', 'rust', 'scab']]).unsqueeze(-1)
            return image, label, self.label_map_reverse[label.squeeze(1).numpy().argmax()]
        else:
            return image

In [ ]:
import pandas as pd
te_df = pd.read_csv('/content/drive/My Drive/test.csv')

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((512, 512)),
        #transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
#     'val': transforms.Compose([
#         transforms.Resize((224,224)),
#         #transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
}

In [ ]:
te_dataset = ppDataset(te_df, '/content/drive/My Drive/Plant_Pathology/test/', return_labels = False, transforms = data_transforms['train'])


In [ ]:
test_loader=torch.utils.data.DataLoader(te_dataset, batch_size=16,
                                             shuffle=False, num_workers=16)

In [ ]:
from PIL import Image

In [ ]:
def get_all_predictions(loaded_data,my_network):
  my_network.to(device)
  final_stacked_result=None
  for batch in (loaded_data):
    with torch.set_grad_enabled(False):
         predictions=my_network(batch.to(device))
         test_prediction=predictions.data.cpu()
         if final_stacked_result is None:
            final_stacked_result=test_prediction.data.cpu()
         else:
             final_stacked_result=torch.cat((final_stacked_result,test_prediction.data.cpu()),dim=0)
  return final_stacked_result

#calling the function 
my_predictions=get_all_predictions(test_loader,model_ft)

In [ ]:
my_predictions.shape

torch.Size([1821, 4])

In [ ]:
my_predictions[1]

tensor([-1.6185,  0.7734,  3.4438, -3.5662])

In [ ]:
my_test_csv1=pd.read_csv('/content/test.csv')

In [ ]:
my_test_csv1['healthy']=np.array(my_predictions[:,0])
my_test_csv1['multiple_diseases']=np.array(my_predictions[:,1])
my_test_csv1['rust']=np.array(my_predictions[:,2])
my_test_csv1['scab']=np.array(my_predictions[:,3])

In [ ]:
my_test_csv1.to_csv('predicted_test_csv.csv',index=False)